## Car Price Prediction [ Data Cleaning and Preparation]
There are numerous factors to consider when deciding how much to pay to buy or sell a car. We can explore the data collected from the online marketplaces listed below, to answer questions that may influence our decision when deciding to buy and sell a car, such as which car brand is the most popular and valued in the region, and finally, we can build a machine learning model to estimate how much to price a car.

Data was collected from the two marketplaces listed below:
- [cars45](https://www.cars45.com/)
- [autochek.africa](https://autochek.africa/ng/cars-for-sale)  

And this notebook details the steps performed in order to properly clean and prepare the data for data exploration and model building.

In [1]:
# importing required packages
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# reading in the cars dataset
autochek_df = pd.read_csv("data/autochek.csv")
cars45_df = pd.read_csv("data/cars45.csv")

##  Understanding the data
The first step in data cleaning and preparation is to understand the data. Here,  it's important to get a feel for the data by examining a sample of the records and going through each of the columns to understand what kind of values are allowed and stored in each of the columns.

In [3]:
## we have 2526 records
print(f"Number of records in autochek dataset: {len(autochek_df)}")
print(f"Number of records in cars45 dataset: {len(cars45_df)}")

Number of records in autochek dataset: 696
Number of records in cars45 dataset: 1830


In [4]:
## Starting data cleaning firstly with the autochek dataset
autochek_df.head()

,Unnamed: 0,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Exterior Color,Ratings
0,0,Toyota,Toyota Corolla,2015,"186,024",Foreign Used,"₦ 7,530,000",4-Cylinder(I4),Approx. 1000,Grey,(4.20)
1,1,Mercedes-Benz,Mercedes-Benz GLK 350,2015,"87,465",Foreign Used,"₦ 13,530,000",6-Cylinder(V6),Approx. 3,Black,(4.50)
2,2,Toyota,Toyota Highlander,2008,"142,719",Foreign Used,"₦ 7,230,000",6-Cylinder(V6),Approx. 3500,Red,(4.40)
3,3,Toyota,Toyota Sienna,2006,"115,232",Foreign Used,"₦ 4,880,000",6-Cylinder(I6),Approx. 3,Gold,(4.50)
4,4,Hyundai,Hyundai Grand Santa Fe,2013,"110,903",Foreign Used,"₦ 9,015,000",6-Cylinder(V6),Approx. 3300,Silver,(4.00)


In [5]:
[## printing out the columns information]
autochek_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       696 non-null    int64 
 1   Brand            696 non-null    object
 2   Model            696 non-null    object
 3   Year             696 non-null    int64 
 4   Mileage          696 non-null    object
 5   Condition        696 non-null    object
 6   Price            696 non-null    object
 7   Engine Type      696 non-null    object
 8   Engine Capacity  696 non-null    object
 9   Exterior Color   696 non-null    object
 10  Ratings          540 non-null    object
dtypes: int64(2), object(9)
memory usage: 59.9+ KB


## Data Cleaning
Data cleaning is one of the most essential subtasks in any data science project.  
Looking at the subset of the data above, we can see that a few of the columns, such as **Mileage, price**, and **Ratings** containing special characters such as [ ₦, commas, whitespaces, and brackets], which we will proceed to remove.
We also drop the redundant **Unnamed: 0** column.

In [6]:
# droping the Unnamed:0 columns
autochek_df = autochek_df.drop('Unnamed: 0',axis=1)

In [7]:
## removing the brackets in the ratings columns
autochek_df['Ratings'] = autochek_df['Ratings'].str.replace("[()]",'',regex=True)
autochek_df.head()

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Exterior Color,Ratings
0,Toyota,Toyota Corolla,2015,"186,024",Foreign Used,"₦ 7,530,000",4-Cylinder(I4),Approx. 1000,Grey,4.20
1,Mercedes-Benz,Mercedes-Benz GLK 350,2015,"87,465",Foreign Used,"₦ 13,530,000",6-Cylinder(V6),Approx. 3,Black,4.50
2,Toyota,Toyota Highlander,2008,"142,719",Foreign Used,"₦ 7,230,000",6-Cylinder(V6),Approx. 3500,Red,4.40
3,Toyota,Toyota Sienna,2006,"115,232",Foreign Used,"₦ 4,880,000",6-Cylinder(I6),Approx. 3,Gold,4.50
4,Hyundai,Hyundai Grand Santa Fe,2013,"110,903",Foreign Used,"₦ 9,015,000",6-Cylinder(V6),Approx. 3300,Silver,4.00


In [8]:
# removing the naira currency and commas from the Price and Mileage column
autochek_df = autochek_df.assign(
    Price = autochek_df["Price"].str.replace("[₦,]",'',regex=True),
    Mileage = autochek_df["Mileage"].str.replace(",",'',)
    )
autochek_df.head()

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Exterior Color,Ratings
0,Toyota,Toyota Corolla,2015,186024,Foreign Used,7530000,4-Cylinder(I4),Approx. 1000,Grey,4.20
1,Mercedes-Benz,Mercedes-Benz GLK 350,2015,87465,Foreign Used,13530000,6-Cylinder(V6),Approx. 3,Black,4.50
2,Toyota,Toyota Highlander,2008,142719,Foreign Used,7230000,6-Cylinder(V6),Approx. 3500,Red,4.40
3,Toyota,Toyota Sienna,2006,115232,Foreign Used,4880000,6-Cylinder(I6),Approx. 3,Gold,4.50
4,Hyundai,Hyundai Grand Santa Fe,2013,110903,Foreign Used,9015000,6-Cylinder(V6),Approx. 3300,Silver,4.00


## Cleaning the Brand Column
Looking at specific brand names, we can certain names appearing as shortened form of another, in which we need to properly replace into the full form

In [9]:
autochek_df["Brand"].value_counts()

Toyota           337
Lexus             74
Mercedes-Benz     60
Ford              29
Kia               27
Honda             25
Hyundai           18
IVM               15
Suzuki            14
JAC               12
Mitsubishi        11
Renault           11
Foton              7
DONGFENG           6
Chevrolet          6
Land               6
Acura              6
Nissan             5
Jeep               3
Fiat               3
Pontiac            3
Mazda              3
MINI               3
Volkswagen         2
Audi               2
Dodge              2
Volvo              1
Infiniti           1
Chrysler           1
BMW                1
Jaguar             1
GEELY              1
Name: Brand, dtype: int64

In [10]:
## Checking the rows with the Land brand, we can see the brand was shortened from Land Rover and
#the Models needs to be correctly edited as Range Rover 
autochek_df[autochek_df["Brand"]=='Land']

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Exterior Color,Ratings
103,Land,Land Rover Range Rover,2014,44492,Foreign Used,35030000,6-Cylinder(V6),Approx. 1500,White,4.40
106,Land,Land Rover range rover Hse,2013,150599,Foreign Used,30015000,8-Cylinder(V8),Approx. N/A,Black,4.30
317,Land,Land Rover Range Rover,2011,123648,Foreign Used,14015000,8-Cylinder(V8),Approx. 1000,Dark Blue,4.40
320,Land,Land Rover Range Rover,2018,18181,Locally used,65605000,8-Cylinder(V8),Approx. 5,Black,3.50
496,Land,Land Rover range rover Hse,2018,44547,Foreign Used,64515000,6-Cylinder(V6),Approx. 6,Black,4.50
655,Land,Land Rover Range Rover,2014,72556,Locally used,23015000,6-Cylinder(I6),Approx. 6000,Black,0.00


In [11]:
autochek_df.loc[autochek_df['Brand']=='Land','Model']='Land Range Rover'## The Land will be removed in the next step
autochek_df.loc[autochek_df['Brand']=='Land','Brand']='Land Rover'

In [12]:
autochek_df[autochek_df['Brand']=='Land Rover']

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Exterior Color,Ratings
103,Land Rover,Land Range Rover,2014,44492,Foreign Used,35030000,6-Cylinder(V6),Approx. 1500,White,4.40
106,Land Rover,Land Range Rover,2013,150599,Foreign Used,30015000,8-Cylinder(V8),Approx. N/A,Black,4.30
317,Land Rover,Land Range Rover,2011,123648,Foreign Used,14015000,8-Cylinder(V8),Approx. 1000,Dark Blue,4.40
320,Land Rover,Land Range Rover,2018,18181,Locally used,65605000,8-Cylinder(V8),Approx. 5,Black,3.50
496,Land Rover,Land Range Rover,2018,44547,Foreign Used,64515000,6-Cylinder(V6),Approx. 6,Black,4.50
655,Land Rover,Land Range Rover,2014,72556,Locally used,23015000,6-Cylinder(I6),Approx. 6000,Black,0.00


### Cleaning the model column
By examining the **Model** column, we can see the name of the brand included with the model name, so we proceed to drop that from the column.

In [13]:
# removing the Brand name from the model
autochek_df['Model'] = autochek_df['Model'].apply(lambda x:" ".join(x.split()[1:]))
autochek_df['Model']

0                                   Corolla
1                                   GLK 350
2                                Highlander
3                                    Sienna
4                            Grand Santa Fe
                       ...                 
691                                   Camry
692                                 Corolla
693                                  IS 350
694                                  RX 350
695    Gratour PX Van - 1.2L Petrol, 750kgs
Name: Model, Length: 696, dtype: object

In [14]:
## No changes is needed here since the column is a categorical variable
autochek_df['Engine Type']= autochek_df['Engine Type'].apply(lambda x:x[0])
autochek_df['Engine Type']

0      4
1      6
2      6
3      6
4      6
      ..
691    4
692    4
693    6
694    6
695    4
Name: Engine Type, Length: 696, dtype: object

In [15]:
autochek_df['Engine Capacity'].value_counts().head(10)

Approx. N/A     286
Approx. 1000     87
Approx. 2500     46
Approx. 3500     39
Approx. 2        27
Approx. 3        25
Approx. 1500     21
Approx. 4        19
Approx. 6        18
Approx. 2000     18
Name: Engine Capacity, dtype: int64

### Cleaning and Preparing the Engine Capacity column
The **Engine Capacity** column has a few issues that need to be addressed, such as the fact that each value is preceded by the string '**Approx.**' and null values are also encoded as 'N/A'.We proceeded to remove the '**Approx.**' keyword before every value, replacing the null values encoded as 'N/A' with an actual null value.  
We also see values such as 1,150,000,  which we will edit since we assume the sellers are using different units to represent the engine capacity since most car engines are in the range of a couple thousands in milliliters.

In [16]:
## checking the engine capacity column
autochek_df['Engine Capacity'] = autochek_df['Engine Capacity'].str.replace("Approx. ",'',regex=True)
autochek_df['Engine Capacity'].value_counts().head(10)

N/A     286
1000     87
2500     46
3500     39
2        27
3        25
1500     21
4        19
6        18
2000     18
Name: Engine Capacity, dtype: int64

In [17]:
# most cars has an engine capacity between 1000 - 6000 ml so we can convert every values to be in this range
def normalize(x):
    if x == 'N/A' or x== np.nan:return np.nan
    x = int(x)
    op = math.floor(1000/x)
    if op>100 and op<= 1000:
        return x * 1000
    elif op>9 and op <=100:
        return x * 100
    elif op>1 and op <=9:
        return x * 10
    elif x//1000>=10 and x//1000 < 100:
        return x/10
    elif x//1000>=100 and x//1000 < 1000:
        return x/100
    elif x//1000>=1000 and x//1000 < 10000:
        return x/1000
    else: return x
autochek_df['Engine Capacity']= autochek_df['Engine Capacity'].apply(normalize)

In [18]:
## our column has been completely edited to this range
autochek_df['Engine Capacity'].describe()[['max','min']]

max    6000.0
min    1000.0
Name: Engine Capacity, dtype: float64

## Cleaning the Exterior Color column

In [19]:
# multiple value of the same color seems to exists
autochek_df['Exterior Color'].value_counts()

White                            132
Black                            132
Silver                            89
Grey                              58
Red                               51
Blue                              51
Gold                              38
Dark Grey                         28
Dark Gray                         11
Gray                              11
Brown                             11
Ash                                8
Dark Blue                          8
Green                              6
Beige                              6
Dark Green                         5
Sliver                             5
Wine                               5
Maroon                             4
Yellow                             4
Dark Red                           3
Dark Silver                        3
Amy Green                          2
Cream                              2
Purple                             2
Light Blue                         2
Off White                          2
R

In [20]:
def rename_color(color):
    color = color.lower()
    if 'grey' in color  or"gray" in color :return "Grey"
    if "blue" in color: return 'Blue'
    if "green" in color: return "Green"
    if "red" in color :return "Red"
    if "silver" in color: return "Silver"
    if "white"  in color :return "White"
    return color
autochek_df['Color']= autochek_df['Exterior Color'].apply(rename_color)
autochek_df = autochek_df.drop("Exterior Color",axis=1)

In [21]:
autochek_df['Color'].value_counts()

White                            137
black                            132
Grey                             110
Silver                            93
Blue                              64
Red                               56
gold                              38
Green                             13
brown                             11
ash                                8
beige                              6
sliver                             5
wine                               5
yellow                             4
maroon                             4
black and orange                   2
purple                             2
cream                              2
blanc                              1
citrine brown metallic finish      1
orange                             1
golden yellow                      1
Name: Color, dtype: int64

In [22]:
# year 2500 exists in the dataset which is surely an error; since this is only one value we drop it
autochek_df['Year'].describe()[['min','max']]

min    2002.0
max    2500.0
Name: Year, dtype: float64

In [23]:
autochek_df= autochek_df[autochek_df['Year']!=2500]
autochek_df['Year'].describe()[['min','max']]

min    2002.0
max    2022.0
Name: Year, dtype: float64

In [24]:
autochek_df['Condition'].value_counts()

Foreign Used    481
New             156
Locally used     58
Name: Condition, dtype: int64

In [25]:
# remapping the column value 
autochek_df['Condition']=autochek_df['Condition'].map(
    {'Foreign Used':"Foreign Used",'Locally used':"Locally Used","New":"Brand New"})

In [26]:
autochek_df['Condition'].value_counts()

Foreign Used    481
Brand New       156
Locally Used     58
Name: Condition, dtype: int64

In [27]:
autochek_df.head()

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Ratings,Color
0,Toyota,Corolla,2015,186024,Foreign Used,7530000,4,1000.0,4.20,Grey
1,Mercedes-Benz,GLK 350,2015,87465,Foreign Used,13530000,6,3000.0,4.50,black
2,Toyota,Highlander,2008,142719,Foreign Used,7230000,6,3500.0,4.40,Red
3,Toyota,Sienna,2006,115232,Foreign Used,4880000,6,3000.0,4.50,gold
4,Hyundai,Grand Santa Fe,2013,110903,Foreign Used,9015000,6,3300.0,4.00,Silver


## Cars 45 dataset
Having clean the first dataset, we proceed to clean the second dataset and rename the columns to be in agreements

In [28]:
cars45_df.head()

,brand,model,mileage,condition,year,color,engine_size,cylinder,price
0,Lexus,RX 330,213521,Nigerian Used,2004,Silver,3500,NaN,"₦ 2,730,000"
1,Toyota,Highlander,235145,Nigerian Used,2005,Silver,3500,NaN,"₦ 1,890,000"
2,Toyota,Corolla,405472,Nigerian Used,2002,Black,1800,NaN,"₦ 1,050,000"
3,Ford,Edge,290102,Nigerian Used,2008,Gray,3500,NaN,"₦ 1,275,000"
4,Ford,Edge,76565,Nigerian Used,2014,Black,3500,NaN,"₦ 3,675,000"


In [29]:
# there is no problem with the year column
cars45_df['year'].describe()[['min','max']]

min    1992.0
max    2020.0
Name: year, dtype: float64

In [30]:
cars45_df['condition'].value_counts()

Nigerian Used    1388
Foreign Used      441
Brand New           1
Name: condition, dtype: int64

In [31]:
cars45_df['condition']=cars45_df['condition'].map(
    {'Foreign Used':"Foreign Used",'Nigerian Used':"Locally Used","Brand New":"Brand New"})

In [32]:
cars45_df['color']= cars45_df['color'].apply(rename_color)

In [33]:
cars45_df['color'].value_counts()

black         495
Silver        330
Grey          266
Blue          196
White         176
Red           126
gold          112
Green          71
brown          28
burgandy        9
purple          8
beige           6
ivory           3
matt black      2
orange          1
teal            1
Name: color, dtype: int64

In [34]:
cars45_df['engine_size'].describe()[['min','max']]

min       1100.0
max    2400000.0
Name: engine_size, dtype: float64

In [35]:
cars45_df['engine_size']=cars45_df['engine_size'].apply(normalize)

In [36]:
cars45_df['engine_size'].describe()[['min','max']]

min    1100.0
max    6200.0
Name: engine_size, dtype: float64

In [37]:
cars45_df['cylinder'].value_counts()

6.0    139
4.0    131
8.0     24
Name: cylinder, dtype: int64

In [38]:
cars45_df['price'] = cars45_df["price"].replace("[₦,]",'',regex=True)

In [39]:
cars45_df.head()

,brand,model,mileage,condition,year,color,engine_size,cylinder,price
0,Lexus,RX 330,213521,Locally Used,2004,Silver,3500.0,NaN,2730000
1,Toyota,Highlander,235145,Locally Used,2005,Silver,3500.0,NaN,1890000
2,Toyota,Corolla,405472,Locally Used,2002,black,1800.0,NaN,1050000
3,Ford,Edge,290102,Locally Used,2008,Grey,3500.0,NaN,1275000
4,Ford,Edge,76565,Locally Used,2014,black,3500.0,NaN,3675000


In [40]:
autochek_df.head()

,Brand,Model,Year,Mileage,Condition,Price,Engine Type,Engine Capacity,Ratings,Color
0,Toyota,Corolla,2015,186024,Foreign Used,7530000,4,1000.0,4.20,Grey
1,Mercedes-Benz,GLK 350,2015,87465,Foreign Used,13530000,6,3000.0,4.50,black
2,Toyota,Highlander,2008,142719,Foreign Used,7230000,6,3500.0,4.40,Red
3,Toyota,Sienna,2006,115232,Foreign Used,4880000,6,3000.0,4.50,gold
4,Hyundai,Grand Santa Fe,2013,110903,Foreign Used,9015000,6,3300.0,4.00,Silver


renaming the columns of both dataset to be aligned

In [41]:
cars45_df = cars45_df.rename(columns={'brand':"Brand","model":"Model",'mileage':"Mileage",'year':"Year",'color':"Color",
                         "engine_size":"Engine Capacity",'condition':"Condition","price":"Price",'cylinder':"Number of Cylinders"})
autochek_df = autochek_df.rename(columns={"Engine Type":"Number of Cylinders"});

## Combining the two datasets
After cleaning and preparing the two datasets, we proceed to combine the two datasets together and export the data.

In [42]:
## combining both dataset into one
df = pd.concat([autochek_df,cars45_df])

In [43]:
# remove all records with models that occurs only once in the dataset
df = df[df.groupby('Model')['Model'].transform(len) > 2]
len(df)

2205

In [44]:
df.head()

,Brand,Model,Year,Mileage,Condition,Price,Number of Cylinders,Engine Capacity,Ratings,Color
0,Toyota,Corolla,2015,186024,Foreign Used,7530000,4,1000.0,4.20,Grey
1,Mercedes-Benz,GLK 350,2015,87465,Foreign Used,13530000,6,3000.0,4.50,black
2,Toyota,Highlander,2008,142719,Foreign Used,7230000,6,3500.0,4.40,Red
3,Toyota,Sienna,2006,115232,Foreign Used,4880000,6,3000.0,4.50,gold
5,Honda,Crosstour,2012,120661,Foreign Used,5030000,6,3000.0,4.50,Silver


In [45]:
df.to_csv("data/cars_cleaned.csv",index=False)

 This is the end of our data cleaning and preparation